# Spark

Hoewel het MapReduce algoritme van Hadoop een aantal voordelen heeft. 
De meest beperkende eigenschap van het MapReduce algoritme is de snelheid.
Omdat alles ingelezen wordt vanaf de harde schijf, tussenresultaten op de schijf opgeslagen worden en de finale resultaten ook wordt er tot wel 90% van de rekentijd gespendeerd in lees- of schrijfopdrachten.

Spark is geintroduceerd om dit te versnellen door gebruik te maken van in-memory processing.
Hierdoor is Spark tot 3 keer sneller op grote datasets en tot 100 keer op kleinere datasets.

Het spark framework kan gebruik maken van een externe opslag-locatie voor bestanden bij te houden (zoals HDFS) en bestaat uit de volgende componenten:
* SparkCore
* Spark SQL
* Spark Streaming
* MLlib
* SparkGraph

Daarnaast zijn er ook verschillende Spark Api's voor verschillende programmeertalen zoals Python, Scala, Java, ...
Hierdoor is het framework ook flexibeler dan het standaard MapReduce algoritme.
Heel veel informatie over het spark framework vind je in de [documentatie](https://spark.apache.org/docs/latest/quick-start.html) en de programming guides (bovenaan).

In [2]:
from hdfs import InsecureClient

In [3]:
map = 'Spark'

client = InsecureClient('http://localhost:9870', user='bigdata')

if client.status(map, strict=False) is None:
    client.makedirs(map)
else:
    # do some cleaning in case anything else than *.txt is present
    for f in client.list(map):
        client.delete(map + '/' + f, recursive=True)

client.upload(map, 'input.txt')
with client.read(map + '/input.txt') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

Hello World,
hello world,
hello world,

Dit is een voorbeeld file om het Wordcount voorbeeld te testen !


## Installatie

Een python implementatie van Spark kan eenvoudig geinstalleerd worden door het volgende commando uit te voeren. 
Dit moet maar eenmalig gebeuren.
Om te kijken of het reeds geinstalleerd is kan je kijken naar de versie van pyspark (indien geinstalleerd). 
Als de versie correct gereturned wordt, dan is het reeds geinstalleerd.

In [1]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.0
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 1.8.0_392
Branch HEAD
Compiled by user ubuntu on 2023-09-09T01:53:20Z
Revision ce5ddad990373636e94071e7cef2f31021add07b
Url https://github.com/apache/spark
Type --help for more information.


In [4]:
!pip install pyspark
# dit moet je niet uitvoeren, is reeds geinstalleerd

Spark kan op drie manieren werken:
* Boven op MapReduce (traag)
* Boven op Yarn
* Via zijn eigen resource manager

In deze notebook gaan we gebruik maken van Spark gebruikmakende van yarn.   

## Resilient Distributed Datasets



Voor de configuratie moeten we vooral twee zaken aangeven, namelijk:
* Naam van de applicatie (is zichtbaar in de yarn)
* Master url. De url dat het type cluster en hoe het te bereiken aangeeft. Wij gaan vooral werken met local om te communiceren met het lokale bestandssysteem 

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/19 09:22:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Wordcount voorbeeld

Om de api van pyspark te leren kennen kan je gaan naar de [documentatie](https://spark.apache.org/docs/latest/api/python/reference/index.html).
Een eerder stap bij stap uitleg kan je [hier](https://spark.apache.org/docs/latest/api/python/getting_started/index.html) vinden.

In onderstaande code gaan we stap voor stap het wordcount-voorbeeld uitwerken.

Eerst moet er een pyspark context aangemaakt worden als volgt.

In [49]:
# bestand uitlezen
df = spark.read.text('/user/bigdata/Spark/input.txt')

In [8]:
df.count()
df.first()

# filter op de rijen die Hello bevatten
# show print het volledige resultaat
df.filter(df.value.contains('Hello')).show()
# collect brengt alle data lokaal van het gedistribueerde dataframe
# collect gebruik je liefst zo weinig mogelijk om te grote datasets te downloaden te vermijden
df.filter(df.value.contains('Hello')).collect()

+------------+
|       value|
+------------+
|Hello World,|
+------------+



In [13]:
# met withColumn kan je kolommen voor bewerkingen gaan toevoegen
df = df.withColumn('contains', df.value.contains('Hello'))
df.show()

# alternatief voor in pandas
# df["contains"] = df.value.contains('Hello')

# immuatable -> onveranderbaar dus worden nieuwe object aangemaakt

+--------------------+--------+
|               value|contains|
+--------------------+--------+
|        Hello World,|    true|
|        hello world,|   false|
|        hello world,|   false|
|                    |   false|
|Dit is een voorbe...|   false|
+--------------------+--------+



In [31]:
df.rdd.flatMap(lambda line: str(line).split(" ")).map(lambda w: (w,1)).collect()

[("Row(value='Hello", 1),
 ("World,')", 1),
 ("Row(value='hello", 1),
 ("world,')", 1),
 ("Row(value='hello", 1),
 ("world,')", 1),
 ("Row(value='')", 1),
 ("Row(value='Dit", 1),
 ('is', 1),
 ('een', 1),
 ('voorbeeld', 1),
 ('file', 1),
 ('om', 1),
 ('het', 1),
 ('Wordcount', 1),
 ('voorbeeld', 1),
 ('te', 1),
 ('testen', 1),
 ("!')", 1)]

In [50]:
words = df.rdd.flatMap(lambda line: str(line).split(' '))    # met .rdd zetten we een dataframe om naar zijn rdd
# flatMap -> een soort map operatie
# Hello World,     ->      ['Hello', 'World,']      (map operatie)

#flatMap doet het volgende
    # Hello World,
# omzetten naar
    # Hello
    # World, 
words_mapped = words.map(lambda w: (w,1))            # map-functie om per entry een 1-waarde toe te voegen als value
words_reduced = words_mapped.reduceByKey(lambda a,b: a+b)   # tel alle waarden per key op

words_reduced.collect()

[("Row(value='Hello", 1),
 ("World,')", 1),
 ("Row(value='hello", 2),
 ("world,')", 2),
 ("Row(value='')", 1),
 ("Row(value='Dit", 1),
 ('is', 1),
 ('een', 1),
 ('voorbeeld', 2),
 ('file', 1),
 ('om', 1),
 ('het', 1),
 ('Wordcount', 1),
 ('te', 1),
 ('testen', 1),
 ("!')", 1)]

**Wat gebeurt er in dit voorbeeld?**

Sparkcontext om een connectie te maken met de distributed storage
De input file wordt dan ingelezen met de textFile functie.
Door middel van de flatMap functie wordt de tekst lijn per lijn ingelezen en gesplits in woorden. 
Dit resulteert in een RDD (Resilient Distributed Dataset.
De .map() functie maakt een key-value pair aan voor elke keer dat het woord voorkomt.
In een laatste fase is er een reduce stap per key die de som neemt van alle keren dat het woord voorkomt om de uiteindelijke wordcount te nemen.
Of af te ronden wordt het resultaat opgeslagen.

![spark wordcount in yarn](images/yarn_001.png)

## SparkSession

Nu gaan we stuk voor stuk de verschillende stappen bekijken om een pyspark applicatie te maken.
De eerste stap is het aanmaken van een sessie (SparkSession) wat het beginpunt is voor spark applications.
Er zijn twee manieren om een SparkSession aan te maken:
* builder()
* newSession()

Bij het aanmaken van een session wordt er intern een SparkContext object aangemaakt. 
Dit object stelt de connectie naar een cluster voor.
Er kan maar 1 context tegelijkertijd actief zijn.
Als je wil connecteren met een tweede cluster moet je eerst stop() oproepen op de reeds actieve context.

In [42]:
spark.stop() # stop de huidige sparksession

#.master("local[2]")         # run lokaal met twee cores
spark = SparkSession.builder.master("yarn").appName("Word Count").getOrCreate()   # get or create -> singleton constructie

24/03/19 09:58:23 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServ

## RDD

Op basis van het SparkSession object is het dan mogelijk om RDD-objecten aan te maken.
Een RDD is de basis dataobject binnen Spark dat in parallel op verschillende nodes binnen een cluster kan uitgevoerd worden.
Alle dataobjecten binnen spark horen tot deze klassen en dus zijn er veel mogelijkheden om RDD's aan te maken.
Hier haal ik er twee aan:
* parallelize() om bestaande python objecten om te zetten naar een RDD
* textFile() of andere read methoden om bestanden op de cluster uit te lezen

In [44]:
# voorbeeld van parallelize 
dataList = [("Student1", 8), ("Student2", 16), ("Student3", 11)]
rdd=spark.sparkContext.parallelize(dataList)

# voorbeeld van inlezen van een file
#rdd2 = spark.read.text("/user/bigdata/Spark/input.txt")
#rdd2 = spark.sparkContext.textFile("/user/bigdata/Spark/input.txt")

Met bovenstaande methoden hebben we twee rdd's aangemaakt. 
Op deze objecten kunnen nu verscheidene operaties uitgevoerd worden.
Een belangrijke eigenschap van dit type objecten is dat ze steeds in parallel uitgevoerd worden.

De beschikbare operaties kunnen in twee groepen verdeeld worden:
* transformaties
* acties

[Transformaties](https://sparkbyexamples.com/apache-spark-rdd/spark-rdd-transformations/) zijn lazy-operations waarvoor de berekening uitgesteld wordt en geven een nieuw RDD terug.
Een aantal voorbeelden van transformaties zijn:
* flatMap()
* map()
* reduceByKey()
* filter()
* sortByKey()

[Acties](https://sparkbyexamples.com/apache-spark-rdd/spark-rdd-actions/) zijn operaties die een berekening starten (ook van de nodige transformaties) en geven een niet RDD-object terug. 
Een aantal voorbeelden hiervan zijn:
* count()
* collect()
* first()
* max()
* reduce()

Lees nu bovenstaande links en geef de functies die nodig zijn voor de volgende vragen op te lossen. Geef ook aan of het transformaties zijn of acties:
* Het aantal keer dat elke waarde aanwezig is in de dataset (1 functie voor wordcount uit te voeren)
* Uitfilteren van rijen
* Groeperen van een aantal rijen op basis van een bepaalde waarde.
* Toevoegen van een kolom aan elke key (bvb de lengte van een woord)
* Hoe doe je head() uit pandas op RDD's?
* Hoe doe je de apply() uit pandas op RDD's?

Maak nu een spark applicatie dat van de eerste RDD (met de studenten) telt hoeveel studenten geslaagd zijn.

In [45]:
rdd.filter(lambda student: student[1] >=10).count()

2

De applicatie voor het berekenen van een gemiddelde is iets complexer.
Dit soort applicaties kan geschreven worden als volgt:

In [47]:
aantal = rdd.count()

rdd.map(lambda x: x[1]).reduce(lambda x,y: (x+y))/ aantal

11.666666666666666

Schrijf nu een mapreduce applicatie in spark om de tweede RDD van de input te verwerken en het aantal woorden van elke lengte te bekomen.
Tussenresultaten kan je tonen om te debuggen met de .collect() functie. Dit haalt de dataset uit het gedistribueerde Spark geheugen.

In [51]:
words = df.rdd.flatMap(lambda line: str(line).split(' '))   
words_mapped = words.map(lambda w: (len(w),1))            
words_reduced = words_mapped.reduceByKey(lambda a,b: a+b)   

words_reduced.collect()

[(16, 3), (8, 3), (13, 1), (14, 1), (2, 3), (3, 3), (9, 3), (4, 1), (6, 1)]

## Dataframes

Een belangrijke subklasse van RDD's zijn dataframes.
Dit is een veel gebruikte manier om gestructureerde data voor te stellen.
Dataframes in spark is sterk gerelateerd aan de dataframes gezien in pandas.
Het belangrijskte verschil is dat ze verdeeld worden over de cluster en operaties op de dataframes in parallel uitgevoerd worden.
Dataframes kunnen aangemaakt worden door gebruik te maken van de createDataFrame functie in context of ingelezen worden vanuit csv's of jsons. Ten slotte kunnen dataframes ook komen van externe bronnen zoals databases als resultaat van een sql-query.

In [52]:
data = [('Harry', 'Potter','1980-07-31','M',100000000),
  ('Ronald','Wemel','1980-04-01','M',10),
  ('Hermelijn','Griffel','1979-09-19','F',4000)
]

columns = ["firstname","lastname","dob","gender","budget"]
df = spark.createDataFrame(data=data, schema = columns)
df.show()
df.describe().show()

+---------+--------+----------+------+---------+
|firstname|lastname|       dob|gender|   budget|
+---------+--------+----------+------+---------+
|    Harry|  Potter|1980-07-31|     M|100000000|
|   Ronald|   Wemel|1980-04-01|     M|       10|
|Hermelijn| Griffel|1979-09-19|     F|     4000|
+---------+--------+----------+------+---------+



24/03/19 10:29:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+--------+----------+------+-------------------+
|summary|firstname|lastname|       dob|gender|             budget|
+-------+---------+--------+----------+------+-------------------+
|  count|        3|       3|         3|     3|                  3|
|   mean|     NULL|    NULL|      NULL|  NULL|         3.333467E7|
| stddev|     NULL|    NULL|      NULL|  NULL|5.773386936614157E7|
|    min|    Harry| Griffel|1979-09-19|     F|                 10|
|    max|   Ronald|   Wemel|1980-07-31|     M|          100000000|
+-------+---------+--------+----------+------+-------------------+



## PySpark SQL

Bovenstaande datastructuren (RDD's en Dataframes) zijn een onderdeel van het Pyspark sql module.
De Spark API heeft een hele reeks methoden en functies om deze in te laden, uit te lezen en te manipuleren.
Daarnaast maakt deze module het ook mogelijk om SQL-queries uit te voeren op dataframes.
Om SQL-queries uit te voeren op dataframes moet er eerst een view gemaakt worden in het dataframe met de functie createOrReplaceTempView("view_name")

Daarna kan je gebruik maken van de .sql() functie om allerhande sql queries uit te voeren.

In [53]:
df.createOrReplaceTempView("test")    # om een fictieve aan te maken waar je je sql-query op kan uitvoeren
df_male = spark.sql("select * from test where gender='M'")
df_male.show()
df_male2 = df.filter(df.gender == "M")
df_male2.show()

df_num_gender = spark.sql('select gender, count(*) from test group by gender')
df_num_gender.show()
df.groupby("gender").count().show()

df.select("firstname", "lastname").show()

+---------+--------+----------+------+---------+
|firstname|lastname|       dob|gender|   budget|
+---------+--------+----------+------+---------+
|    Harry|  Potter|1980-07-31|     M|100000000|
|   Ronald|   Wemel|1980-04-01|     M|       10|
+---------+--------+----------+------+---------+

+---------+--------+----------+------+---------+
|firstname|lastname|       dob|gender|   budget|
+---------+--------+----------+------+---------+
|    Harry|  Potter|1980-07-31|     M|100000000|
|   Ronald|   Wemel|1980-04-01|     M|       10|
+---------+--------+----------+------+---------+

+------+--------+
|gender|count(1)|
+------+--------+
|     F|       1|
|     M|       2|
+------+--------+

+------+-----+
|gender|count|
+------+-----+
|     F|    1|
|     M|    2|
+------+-----+

+---------+--------+
|firstname|lastname|
+---------+--------+
|    Harry|  Potter|
|   Ronald|   Wemel|
|Hermelijn| Griffel|
+---------+--------+



Buiten de functionaliteit om SQL queries uit te voeren is ook het lezen en schrijven van allerhande dataformaten een belangrijk onderdeel van de pyspark sql module.
Meer informatie hierover kun je [hier](https://spark.apache.org/docs/latest/sql-data-sources.html) vinden in de documentatie.
In essentie ziet de code er uit als volgt:

De opties die hierbij gekozen kunnen worden kun je vinden in de documentatie.

Daarnaast zijn er ook functionaliteiten om data uit te lezen speciaal voor Machine Learning zoals libsvm en image-directories maar die worden later getoond.

### Oefening

Net zoals RDD kunnen er een aantal operaties uitgevoerd worden op deze dataframes.
Een volledige lijst met alle operaties kan je [hier](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#dataframe-apis) vinden.
Zoek de functies die gebruikt moeten worden om de volgende zaken uit te voeren:
* Groepeer volgens een bepaalde sleutel
* Krijg een lijst met alle kolomnamen
* Filter rijen uit
* Verwijder null-values in de dataset via rijen
* Verwijder null-values door kolommen te verwijderen
* Bereken een dataframe met statistieken van het dataframe
* Krijg een dataframe met alle nan waarden
* Hoe krijg je informatie zoals .info()
* Hoe werkt het groeperen van informatie op basis van een key/kolom

Probeer deze ook uit op bovenstaand aangemaakt dataframe

Lees daarna volgende [link](https://sparkbyexamples.com/pyspark/pyspark-aggregate-functions/) om een idee te krijgen over hoe verschillende functies uit te voeren op deze dataframes.
Werk nu de volgende oefening uit en maak hiervoor een spark applicatie:
* Download de iris dataset in sklearn
* Bewaar deze dataset als csv en upload de file naar het hdfs
* Schrijf de code om de csv uit te lezen en om te zetten naar een dataframe
* Print het dataschema uit voor het dataframe, hoeveel kolommen zijn er aanwezig in het dataframe.
* Bereken het minimum en maximum van de 'sepal width (cm)' en 'petal width (cm)' kolom.
* Hernoem de target kolom naar label
* Hernoem de labels 0 naar Soort 0, labels 1 naar Soort 2 en labels 3 naar Soort 3
* Voer normalisatie van de eerste 4 kolommen uit (het gemiddelde ervan aftrekken en delen door de standaardafwijking)
* Controleer de voorgaande stap door opnieuw het gemiddelde en de standaardafwijking te berekenen. Deze moeten respectievelijk 0 en 1 zijn.
* Bereken de oppervlakte van sepal door de lengte en breedte ervan te vermenigvuldigen. Noem deze nieuwe kolom sepal area. Doe dit ook voor de petal.
* Groepeer nu de rijen op de label kolom. Bereken per groep het gemiddelde van elke kolom. Is er een verschil tussen de verschillende klassen?

**Shared variabelen**

Variabelen met read-write acces zijn zeer inefficient om te gebruiken in een cluster met sterke parallelisatie.
Spark bied echter twee varianten aan die wel efficient geimplementeerd kunnen worden, namelijk
* Broadcasted variabelen
* Accumulators

Broadcasted variabelen zijn read-only variabelen, die aangemaakt worden door de driver en eenmalig verspreid worden over de nodes in plaats van voor elke job.
Dit wordt vooral gebruikt om grote data die veelvuldig gebruikt wordt te cachen op de nodes.
Bij het gebruik van broadcast variabelen is het belangrijk om te onthouden dat je de originele variabele niet meer mag gebruiken na het aanmaken van de broadcasted variabele omdat ze anders toch nog elke job doorgestuurd wordt.
De belangrijkste functies om te werken met broadcasted variabelen zijn:

**Accumulators**

Het andere type dat aangeboden wordt zijn accumulators.
Deze laten enkel toe dat noden iets toevoegen aan een gedeelde variabele.
Enkel de driver kan deze variabele uitlezen.
Dit kan bijvoorbeeld gebruikt worden om tellers of sommen bij te houden.
Deze accumulators kunnen een naam hebben (named accumulators zijn zichtbaar in de wep api).
De ingebouwde accumulator van Spark ondersteunt enkel numerieke accumulators.
Het is echter mogelijk om eigen accumulators toe te voegen door over te erven van de AccumulatorParam klasse en deze twee functies te implementeren:
* zero: De begin waarde van de accumulator
* addInPlace: Om twee waarden samen te voegen

## Pandas

Door de hoge populariteit van pandas in python is er een alternatief uitgewerkt binnen de laatste versie van Spark (eind 2021) dat de pandas api integreert.
Hierdoor kan je code schrijven die identiek is aan te werken met pandas.
Meer informatie over deze api vind je op de [pyspark documentatie](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html).

De volgende twee delen van deze documentatie zijn belangrijk:
* [De beschikbare pandas functies](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/supported_pandas_api.html): Niet alle functies van pandas kunnen gebruikt worden. Dit komt doordat pandas geschreven is voor data die volledig lokaal ingeladen is in het geheugen. Omdat dit gevaarlijk kan zijn in het geval van grote datasets zijn de functies die dit zouden uitvoeren niet geimplementeerd. De meeste functies die we gebruiken zijn gelukkig wel geimplementeerd.
* [De best practices](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/best_practices.html) om te werken met pandas on spark. Samengevat is het:
  * Gebruik de .explain() functie in spark om het plan van uitvoering te bekijken indien het uitvoeren te lang duurt
  * Gebruik checkpoints voor de efficientie van de planner van het stappenplan (spark.checkpoint() of spark.local_checkpoint()
  * **Vermijd shuffling van data zoals sort-operaties**. Hierbij wordt data tussen nodes verstuurd wat kan resulteren in heel wat communicatie tussen verschillende nodes
  * **Pas op met kolomnamen**: Gebruik geen duplicaten en vermijd gereserveerde namen met leading en trailing dubbele underscores
  * **Stel waar mogelijk de index kolom in bij converteren naar een pandas-on-spark dataframe**: Veel functies werken efficienter bij het gebruik van een goede indexwaarde ipv de default.
  * **Vermijd merge/join operaties waar mogelijk**: Ook deze operaties vereisen heel wat communicatie dus worden best vermeden
  * Gebruik zoveel mogelijk van de pandas-on-spark API direct ipv standaard python functies om conflicten te vermijden
     * df.sum() werkt gedistribueerd maar sum(df) niet wat tot fouten en errors leidt

## Verdere oefeningen

Gebruik nu de titanic csv om met behulp van een spark applicatie de volgende zaken te berekenen:
* Het aantal unieke plaatsen waar personen aan boord zijn gekomen (embarked kolom)
* Het aantal ontbrekende waarden in de Cabin kolom
* De volgende statistische waarden voor de ticketprijs (Fare) kolom: min, max, mean, std
* De langste naam van een passagier
* Het aantal passagiers per klasse
* Het totaal aantal passagiers op de titanic

Het is de bedoeling dat je elk element afzonderlijk berekend dus je moet geen dataframe uitkomen waar al deze zaken in zitten. 

Tip: herstart je kernel zodat je het opzetten van de sparkcontext ook oefent. 